# Strava data processing and exploration

This notebook uses the data fetched by the *fetch_data.ipynb* and explores Strava extracted data.

Data are:

- Athlete profile
- All Atlhlete activities with:
    - Base activity data such as distance, average pace, average HR etc...
    - Advanced data - streams - about HR, pace etc ...


---



## Dependencies and configuration

import needed dependencies and configure the environment.

In [1]:
import json
import math
import time
import os
from pathlib import Path
from typing import Dict, Any
import threading
import requests
import polars as pl
import numpy as np

from utils import config

Loaded STRAVA_CLIENT_ID: 176459
Loaded STRAVA_CLIENT_SECRET: 1a9d963bec85c4de91bd28526331a13ef578a524
Loaded STRAVA_REFRESH_TOKEN: b873396b75c9504b9ebd23bac3216e476cb9d2a4
Loaded STRAVA_ACCESS_TOKEN: f4bc0ac3563ab45187161065726ccacb984d8bac
Loaded STRAVA_USER_AUTHORIZATION_CODE: 1f6d477ee870de8113f2fe2f44283124ceca5935


In [ ]:
def load_file(path: Path) -> pl.DataFrame:
    p = str(path)
    # Sniff the first non-whitespace char to choose parser
    try:
        with open(p, 'r', encoding='utf-8') as f:
            head = f.read(2048).lstrip()
    except Exception:
        head = ""

    if head.startswith("["):
        # JSON array of objects
        return pl.read_json(p)

    # Try NDJSON (one JSON object per line) first; fallback to standard JSON
    try:
        return pl.read_ndjson(p)
    except Exception:
        return pl.read_json(p)

## Data Exploration

### 1. Basic athlete data

In [6]:
athlete_profile = load_file(config.ATHLETE_DATA)

print(athlete_profile)

shape: (1, 33)
┌──────────────┬─────────┬────────┬───────────┬───┬──────────┬────────┬──────────────┬─────────────┐
│ resource_sta ┆ country ┆ friend ┆ state     ┆ … ┆ city     ┆ summit ┆ bikes        ┆ shoes       │
│ te           ┆ ---     ┆ ---    ┆ ---       ┆   ┆ ---      ┆ ---    ┆ ---          ┆ ---         │
│ ---          ┆ str     ┆ null   ┆ str       ┆   ┆ str      ┆ bool   ┆ list[struct[ ┆ list[struct │
│ i64          ┆         ┆        ┆           ┆   ┆          ┆        ┆ 8]]          ┆ [8]]        │
╞══════════════╪═════════╪════════╪═══════════╪═══╪══════════╪════════╪══════════════╪═════════════╡
│ 3            ┆ France  ┆ null   ┆ Aquitaine ┆ … ┆ Jurançon ┆ true   ┆ [{"b7438340" ┆ [{"g2216790 │
│              ┆         ┆        ┆           ┆   ┆          ┆        ┆ ,false,"Trib ┆ 2",false,"S │
│              ┆         ┆        ┆           ┆   ┆          ┆        ┆ an 520…      ┆ aucony T…   │
└──────────────┴─────────┴────────┴───────────┴───┴──────────┴────────┴─────

### 2. Base activity statistics

Some stats about activities over time

In [7]:
activities = load_file(config.ATHLETE_ACTIVITIES)

print(activities.shape)
print(activities.head())

ComputeError: ExpectedArrayContent at character 2392 ('{')